<a href="https://colab.research.google.com/github/lr0103/data-science/blob/master/MapasCom4SquareAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Working with Maps and 4 Square API**

In this notebook we will see about:


*   4 Square API
*   Maps API
*   Scraping HTML pages



In [1]:
#Importação das libraries
import pandas as pd

###**1 - Data Scraping from HTML Page**

####1.1 -Making HTML page download

In [2]:
!wget -q -O 'canadatapage.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


####1.2 - HTML page Transformation on Pandas's Data Frame

In [10]:
df_html = pd.read_html('canadatapage.html')
df_html[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [11]:
df_canada_raw=df_html[0]

In [12]:
df_canada_raw.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


####1.4 - Cleaning dataset

In [13]:
#Ignore cells with a borough that is Not assigned
df_canada_neighbourhood= df_canada_raw[ df_canada_raw.Borough != 'Not assigned']

In [14]:
df_canada_neighbourhood

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
#If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough
import numpy as np

df_canada_neighbourhood['Neighbourhood']= np.where(df_canada_neighbourhood['Neighbourhood'] != 'Not assigned', df_canada_neighbourhood.Neighbourhood, df_canada_neighbourhood.Borough)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
print('Canda dataframe shape:',df_canada_neighbourhood.shape)

Canda dataframe shape: (103, 3)


#### 1.5 - Getting the latitude and the longitude coordinates of each neighborhood

In [17]:
# Inttalling Geocoder Python package 
!pip install geopy

#####**Use sample of geopy**

import geocoder # import geocoder

#####**initialize your variable to None**
lat_lng_coords = None

#####**loop until you get the coordinates**
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [18]:
# I will not use Geocoder.
# I will use a offline cvs file
!wget -q -O 'coordenates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [19]:
df_coord= pd.read_csv("coordenates.csv")

In [20]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
#Mergging Dataframes
df_canada= pd.merge(df_canada_neighbourhood,df_coord, on='Postal Code')

In [22]:
df_canada

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [23]:
''

''